## Tuning Deep Models  Premier Analysis on Azure Machine Learning
### LSTM and DAN 

In [1]:
import argparse
import os
import sklearn
import pandas as pd 
import numpy as np
from sklearn.metrics import f1_score,accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from azureml.core import Run, Dataset, Environment,Experiment,ScriptRunConfig
from sklearn.preprocessing import LabelEncoder
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import DEFAULT_CPU_IMAGE
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.estimator import Estimator
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive import choice,uniform


In [2]:
from azureml.core import  Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: cdh-azml-dev-mlw
Azure region: eastus
Subscription id: 320d8d57-c87c-4434-827f-59ee7d86687a
Resource group: CSELS-CDH-DEV


In [3]:
# current working directory
path = os.getcwd()
print("Current Directory:", path)
  
# parent directory
parent = os.path.join(path, os.pardir)
  
# prints parent directory
print("\nParent Directory:", os.path.abspath(parent))

premier_path = os.path.abspath(parent)

Current Directory: c:\Users\wsn8\Code\premier_analysis\azure_ml

Parent Directory: c:\Users\wsn8\Code\premier_analysis


### Create Compute

In [4]:
clustername = 'StandardNC6'
is_new_cluster = False
try:
    aml_cluster = ComputeTarget(workspace = ws,name= clustername)
    print("Find the existing cluster")
except ComputeTargetException:
    print("Cluster not find - Creating cluster.....")
    is_new_cluster = True
    compute_config = AmlCompute.provisioning_configuration(vm_size='StandardNC6',
                                                           max_nodes=2)
    aml_cluster = ComputeTarget.create(ws, clustername, compute_config)

aml_cluster.wait_for_completion(show_output=True)

Find the existing cluster
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


### Environment

In [5]:
premier_train_model_env = Environment.from_conda_specification(name='premier_train_model_env', file_path='./environments/conda_dependencies_model.yml')
# Specify a GPU base image
premier_train_model_env.docker.enabled = True
premier_train_model_env.docker.base_image = DEFAULT_CPU_IMAGE
premier_train_model_env.register(workspace=ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04:20210220.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": true,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "premier_train_model_env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "de

### LSTM tuning

In [7]:
OUTCOME = 'icu'
MODEL = 'lstm'
EPOCHS = 5

exp_name = f"Job-tune-deepmodel-all_days-{OUTCOME}-{MODEL}"
print("exp_name:",exp_name)
experiment = Experiment(workspace=ws, name=exp_name)

exp_name: Job-tune-deepmodel-all_days-icu-lstm


In [12]:

# Specify parameter sampler
ps = RandomParameterSampling({
    "--demog":choice('True'),
    "--dropout":uniform(0.0,0.9),
    "--recurrent_dropout": uniform(0.0,0.9),
    "--n_cells":choice(128,256)
})


# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)


est = ScriptRunConfig(source_directory='./training',
                      script='train_model.py',
                      arguments=['--all_days','--outcome', OUTCOME,'--model', MODEL,'--epochs',EPOCHS],
                      compute_target=aml_cluster,
                      environment=premier_train_model_env)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config= est,
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name="val_loss",
                             primary_metric_goal=PrimaryMetricGoal.MINIMIZE,
                             max_total_runs=10,
                             max_concurrent_runs=4)

In [13]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = experiment.submit(hyperdrive_config)
hyperdrive_run

Experiment,Id,Type,Status,Details Page,Docs Page
Job-tune-deepmodel-all_days-icu-lstm,HD_badc70e0-a9d6-4ba4-92b2-0d8e53084c5f,hyperdrive,Running,Link to Azure Machine Learning studio,Link to Documentation


In [14]:
hyperdrive_run.get_status()

'Running'

In [15]:
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

Performing interactive authentication. Please follow the instructions on the terminal.


Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"


### Get best model